In [ ]:
pip install git+https://github.com/facebookresearch/segment-anything.git

In [ ]:
pip install SimpleITK

In [ ]:
import os
import torch
join = os.path.join
import numpy as np
from tqdm import tqdm
import SimpleITK as sitk
import matplotlib.pyplot as plt
from skimage import transform, io, segmentation
from segment_anything import SamPredictor, sam_model_registry
from segment_anything.utils.transforms import ResizeLongestSide
np.random.seed(2023)

In [ ]:
# Data preprocessing and saving numpy arrays of CT scans and gts

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
checkpoint = "/content/drive/MyDrive/Colab Notebooks/MedSAM-Adrenal/SAM/sam_vit_b_01ec64 (1).pth"
sam = sam_model_registry['vit_b'](checkpoint=checkpoint).to(device)

In [ ]:

def window_image(img,rescale=True):
    # img = (img*slope +intercept) #for translation adjustments given in the dicom file.
    window_center = 50
    window_width = 350

    img_min = window_center - window_width//2 #minimum HU level
    img_max = window_center + window_width//2 #maximum HU level
    img[img<img_min] = img_min #set img_min for all HU levels less than minimum HU level
    img[img>img_max] = img_max #set img_max for all HU levels higher than maximum HU level
    if rescale:
        img = (img - img_min) / (img_max - img_min)*255.0
    return img

def preprocess_data(img_path,gt_path,sam):
    # For 1 patient only
    # images and ground truths as numpy are
    # stored in this
    imgs = []
    gts  = []
    img_embeddings = []

    # loading the ground truth maps for
    # both left and right adrenal
    gt_sitk = sitk.ReadImage(gt_path)
    gt_data = sitk.GetArrayFromImage(gt_sitk)
    gt_data = np.logical_or(gt_data == 7, gt_data == 8).astype(np.uint8)

    # loading ground truth data
    # size is N x 512 x 512
    img_sitk = sitk.ReadImage(img_path)
    img_data = sitk.GetArrayFromImage(img_sitk)

    # applying abdomen window to all the N slices of CT
    for i in range(img_data.shape[0]):
        img_data[i] = window_image(img_data[i])
        img_data[i] = img_data[i].astype(np.uint8)

    z_index, _, _ = np.where(gt_data>0)
    z_min, z_max = np.min(z_index), np.max(z_index)

    # traversal over the non zero segmentation maps
    # and corresponding slices
    print("\nPatient slices processing :")
    for i in tqdm(range(z_min,z_max)):
        # should be 512x512
        gt_slice_i = gt_data[i,:,:]
        # resize the gt to 256 as the output of sam is 256
        gt_slice_i = transform.resize(gt_slice_i, (image_size, image_size), order=1, preserve_range=True, mode='constant', anti_aliasing=True)

        # should be 512x512
        img_slice_i = img_data[i,:,:]
        img_slice_i = transform.resize(img_slice_i,(image_size, image_size), order=3, preserve_range=True, mode='constant', anti_aliasing=True)
        img_slice_i = np.uint8(np.repeat(img_slice_i[:,:,None], 3, axis=-1))
        # now should be 3x256x256

        assert len(img_slice_i.shape)==3 and img_slice_i.shape[2]==3, 'image should be 3 channels'
        assert img_slice_i.shape[0]==gt_slice_i.shape[0] and img_slice_i.shape[1]==gt_slice_i.shape[1], 'image and ground truth should have the same size'

        img_slice_i = np.rot90(img_slice_i,k=-1)
        img_slice_i = np.rot90(img_slice_i,k=-1)

        gt_slice_i = np.rot90(gt_slice_i,k=-1)
        gt_slice_i = np.rot90(gt_slice_i,k=-1)

        imgs.append(img_slice_i)
        gts.append(gt_slice_i)

        if sam is not None:
            sam_transform = ResizeLongestSide(sam.image_encoder.img_size)
            resize_img = sam_transform.apply_image(img_slice_i)
            # resized_shapes.append(resize_img.shape[:2])

            resize_img_tensor = torch.as_tensor(resize_img.transpose(2, 0, 1)).to(device)
            # model input: (1, 3, 1024, 1024)
            input_image = sam.preprocess(resize_img_tensor[None,:,:,:]) # (1, 3, 1024, 1024)
            assert input_image.shape == (1, 3, sam.image_encoder.img_size, sam.image_encoder.img_size), 'input image should be resized to 1024*1024'
            # input_imgs.append(input_image.cpu().numpy()[0])
            with torch.no_grad():
                embedding = sam.image_encoder(input_image)
                img_embeddings.append(embedding.cpu().numpy()[0])
    if sam is not None:
        return imgs, gts, img_embeddings
    else:
        return imgs, gts

In [ ]:
image_size = 256
img_name_suffix = '_0000.nii.gz'

# image name is like : FLARE22_Tr_0001_0000.nii.gz
# gt name is like    : FLARE22_Tr_0001.nii.gz

#  image name for a particular gt = gt.split('.nii.gz')[0] + img_name_suffix

patients_CT_data_path = "/content/drive/MyDrive/Colab Notebooks/MedSAM-Adrenal/data/FLARE22Train/images"
patients_seg_data_path =  "/content/drive/MyDrive/Colab Notebooks/MedSAM-Adrenal/data/FLARE22Train/labels"
npz_files_path = '/content/drive/MyDrive/Colab Notebooks/MedSAM-Adrenal/Training_July/data_FLARE22_more_patients/Npz_files'

prefix = 'CT_Abd-Adrenal'
# names = [patients from 1 to 50]

patients_gts = sorted(os.listdir(patients_seg_data_path))
# names is like FLARE22_Tr_0001_0000.nii.gz
print(len(patients_gts))

# cleaning the file names and shuffling and dividing
# patients_gts = [patient_gt for patient_gt  in patients_gts if not os.path.exists(join(npz_files_path, prefix + '_' + patient_gt.split('.nii.gz')[0]+'.npz'))]
# patients_gts = [patient_gt for patient_gt  in patients_gts if os.path.exists(join(npz_files_path, patient_gt.split('.nii.gz')[0] + img_name_suffix))]

np.random.shuffle(patients_gts)

# for i in range(50):
#     print(patients_gts[i])

train_gts = sorted(patients_gts[:int(len(patients_gts)*0.8)])
test_gts = sorted(patients_gts[int(len(patients_gts)*0.8):])

# print(len(test_gts))
# print(len(train_gts))

# paths for saving the npz files
save_path_tr = join(npz_files_path, prefix, 'train')
save_path_ts = join(npz_files_path, prefix, 'test')
os.makedirs(save_path_tr, exist_ok=True)
os.makedirs(save_path_ts, exist_ok=True)


print(len(train_gts))

print("Train data processing:-")
for patient_gt in tqdm(train_gts):
    # patient_gt = FLARE22_Tr_0001.nii.gz
    patient_gt_path = join(patients_seg_data_path,patient_gt)
    patient_image = patient_gt.split('.nii.gz')[0] + img_name_suffix
    patient_image_path = join(patients_CT_data_path,patient_image)

    imgs,gts,embeddings = preprocess_data(patient_image_path,patient_gt_path,sam)

    if len(imgs)>1:
        imgs = np.stack(imgs, axis=0) # (n, 256, 256, 3)
        gts = np.stack(gts, axis=0) # (n, 256, 256)
        embeddings = np.stack(embeddings, axis=0) # (n, 1, 256, 64, 64)

        np.savez_compressed(join(save_path_tr, prefix + '_' + patient_gt.split('.nii.gz')[0]+'.npz'), imgs=imgs, gts=gts, img_embeddings=embeddings)
        # save an example image for sanity check

        idx = np.random.randint(0, imgs.shape[0])
        img_idx = imgs[idx,:,:,:]
        gt_idx = gts[idx,:,:]
        bd = segmentation.find_boundaries(gt_idx, mode='inner')
        img_idx[bd, :] = [255, 0, 0]
        io.imsave(save_path_tr + '.png', img_idx, check_contrast=False)

print(len(train_gts))
print("Test data processing:-")
for patient_gt in tqdm(test_gts):
    # patient_gt = FLARE22_Tr_0001.nii.gz
    # function mai path hi bhejdo
    patient_gt_path = join(patients_seg_data_path,patient_gt)
    patient_image = patient_gt.split('.nii.gz')[0] + img_name_suffix
    patient_image_path = join(patients_CT_data_path,patient_image)

    imgs,gts,embeddings = preprocess_data(patient_image_path,patient_gt_path,sam)

    if len(imgs)>1:
        imgs = np.stack(imgs, axis=0) # (n, 256, 256, 3)
        gts = np.stack(gts, axis=0) # (n, 256, 256)
        embeddings = np.stack(embeddings, axis=0) # (n, 1, 256, 64, 64)
        np.savez_compressed(join(save_path_ts, prefix + '_' + patient_gt.split('.nii.gz')[0]+'.npz'), imgs=imgs, gts=gts)
        # save an example image for sanity check
        # ye hai kya
        idx = np.random.randint(0, imgs.shape[0])
        img_idx = imgs[idx,:,:,:]
        gt_idx = gts[idx,:,:]
        bd = segmentation.find_boundaries(gt_idx, mode='inner')
        img_idx[bd, :] = [255, 0, 0]
        io.imsave(save_path_ts + '.png', img_idx, check_contrast=False)

50
40
Train data processing:-


  0%|          | 0/40 [00:00<?, ?it/s]


Patient slices processing :



  2%|▎         | 1/40 [00:26<17:31, 26.96s/it]


Patient slices processing :


...


 80%|████████  | 8/10 [02:04<00:31, 15.98s/it]


Patient slices processing :



 90%|█████████ | 9/10 [02:21<00:16, 16.36s/it]


Patient slices processing :



100%|██████████| 10/10 [02:39<00:00, 15.95s/it]


In [ ]:
test_path = "/content/drive/MyDrive/Colab Notebooks/MedSAM-Adrenal/Training_July/data_FLARE22_more_patients/Npz_files/CT_Abd-Adrenal/test"
train_path = "/content/drive/MyDrive/Colab Notebooks/MedSAM-Adrenal/Training_July/data_FLARE22_more_patients/Npz_files/CT_Abd-Adrenal/train"

print(len(os.listdir(test_path)))
print(len(os.listdir(train_path)))


10
40


In [ ]:
test_images = 0
train_images = 0
test_labels = 0
train_labels = 0

train_files = os.listdir(train_path)
test_files = os.listdir(test_path)

for file in train_files:
    file_path_train = join(train_path,file)

    imgs_train = np.load(file_path_train)['imgs']
    gts_train = np.load(file_path_train)['gts']

    train_images += imgs_train.shape[0]
    train_labels += gts_train.shape[0]

for file in test_files:
    file_path_test = join(test_path,file)

    imgs_test = np.load(file_path_test)['imgs']
    gts_test = np.load(file_path_test)['gts']

    test_images += imgs_test.shape[0]
    test_labels += gts_test.shape[0]


print(test_images)
print(train_images)
print(test_labels)
print(train_labels)

195
782
195
782


FLARE2022 50 PATIENT DATASET

In [2]:
pip install SimpleITK

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 15.4 MB/s eta 0:00:00


In [4]:
# seeing whole dataset
import os
import SimpleITK as sitk
import numpy as np

n_images = 0
n_labels = 0

image_path = "/content/drive/MyDrive/Colab Notebooks/MedSAM-Adrenal/data/FLARE22Train/images"
label_path = "/content/drive/MyDrive/Colab Notebooks/MedSAM-Adrenal/data/FLARE22Train/labels"

# /content/drive/MyDrive/Colab Notebooks/MedSAM-Adrenal/data/FLARE22Train/images/FLARE22_Tr_0001_0000.nii.gz

images = os.listdir(image_path)

for image in images:
    img_path = os.path.join(image_path,image)
    img_sitk = sitk.ReadImage(img_path)
    img_data = sitk.GetArrayFromImage(img_sitk)

    gt = image.replace("_0000","")
    gt_path = os.path.join(label_path,gt)

    gt_sitk = sitk.ReadImage(gt_path)
    gt_data = sitk.GetArrayFromImage(gt_sitk)
    gt_data = np.logical_or(gt_data == 7, gt_data == 8).astype(np.uint8)

    n_images += img_data.shape[0]
    n_labels += gt_data.shape[0]


print(n_images)
print(n_labels)


# 4794 images and labels are there in the dataset



4794
4794
